In [4]:
import numpy as np
import pandas as pd
from pqp.data.data import Data
from pqp.identification.graph import Graph
from pqp.estimation import MultinomialEstimator
from pqp.identification.estimands import *
from pqp.symbols import *

In [5]:
x, y, z = make_vars("xyz")
df = pd.DataFrame({
    "x": [0, 1, 0, 1, 1.5, 1.5],
    "y": [0, 1, 1, 2, 1.5, 2.5],
    "m": [1, 2, 3, 4, 5, 6],
    "z": [0, 0, 1, 1, 0, 1],
})
data = Data(df)

In [6]:
dist = MultinomialEstimator(data, prior=0.1)
dist.print_log(nested=True)

AttributeError: 'MultinomialEstimator' object has no attribute 'print_log'

In [7]:
causal_estimand = ATE(y, {z: 0}, {z: 1})
causal_estimand

<ATE treatment_vars=[Variable(z)], outcome=y>

In [8]:
x, y, z, m = make_vars("xyzm")

g = Graph([
    y <= [x, z],
    x <= z,
    m <= x,
])

g

Graph([DirectedEdge(start=x, end=m), DirectedEdge(start=z, end=x), DirectedEdge(start=z, end=y), DirectedEdge(start=x, end=y)])

In [9]:
estimand = g.identify(causal_estimand)
estimand.print_log()

AttributeError: 'IdentificationResult' object has no attribute 'print_log'

In [10]:
est = dist.estimate(estimand)
est

EstimationResult(value=-0.4102459016393445)

In [11]:
data.df.x[0]

0.37425

In [12]:
x, y, z = make_vars("xyz")

treatment_condition = [x.val == 1]
control_condition = [x.val == 0]

In [25]:
g = Graph([
    y <= z,
    z <= x,
    y & x
])

In [40]:
df = pd.DataFrame({
    "x": [0, 0, 0, 1, 1, 0],
    "z": [0, 1, 0, 1, 1, 0],
    "y": [0, 1, 0, 1, 1, 0],
})

dist = MultinomialEstimator(df, prior=1)
dist

MultinomialEstimator()

In [41]:
causal_estimand = ATE(y, treatment_condition, control_condition)
print(causal_estimand.expression().to_latex())

E_y \big[ P(y \mid \text{do}(x = 1)) \big] - E_y \big[ P(y \mid \text{do}(x = 0)) \big]


In [42]:
estimand = g.identify(causal_estimand).identified_estimand
print(estimand.to_latex())

E_y \big[ \sum_{z} \big({\sum_{x} \big({P(x) P(x, z, y) \over P(x, z)}\big) P(x = 1, z) \over P(x = 1)}\big) \big] - E_y \big[ \sum_{z} \big({\sum_{x} \big({P(x) P(x, z, y) \over P(x, z)}\big) P(x = 0, z) \over P(x = 0)}\big) \big]


In [43]:
dist.estimate(estimand)

EstimationResult(value=0.4433808167141502)